Assignment 4 \
Part 4: DT-Credit Regression \
Submitted by Group 5 \

Groupmembers:

1. Sajan Kumer Sarker (2111131642) (sajan.sarker@northsouth.edu)
2. Rafsan Jani Chowdhury (2011424642) (rafsan.chowdhury@northsouth.edu)
3. Rosely Mohammad (2014219642) (rosely.mohammad@northsouth.edu)

Task: \
1) Experiment with Decision Trees and XGBoost

2) Understanding of Evaluation metrics



**For Regression problem: Mean-squared error**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBClassifier, XGBRegressor

In [3]:
df = pd.read_csv('/content/drive/MyDrive/CSE445-Assignment/Assignment-4/dataset/DT-Credit.csv')
print(df.shape)
print(df.head())

(400, 11)
    Income  Limit  Rating  Cards  Age  Education  Own Student Married Region  \
0   14.891   3606     283      2   34         11   No      No     Yes  South   
1  106.025   6645     483      3   82         15  Yes     Yes     Yes   West   
2  104.593   7075     514      4   71         11   No      No      No   West   
3  148.924   9504     681      3   36         11  Yes      No      No   West   
4   55.882   4897     357      2   68         16   No      No     Yes  South   

   Balance  
0      333  
1      903  
2      580  
3      964  
4      331  


In [4]:
print('Number of missing values: ')
print(df.isnull().sum())

Number of missing values: 
Income       0
Limit        0
Rating       0
Cards        0
Age          0
Education    0
Own          0
Student      0
Married      0
Region       0
Balance      0
dtype: int64


In [5]:
df = pd.get_dummies(df, columns=['Own', 'Student', 'Married', 'Region'])
print(df.shape)
print(df.head())

(400, 16)
    Income  Limit  Rating  Cards  Age  Education  Balance  Own_No  Own_Yes  \
0   14.891   3606     283      2   34         11      333    True    False   
1  106.025   6645     483      3   82         15      903   False     True   
2  104.593   7075     514      4   71         11      580    True    False   
3  148.924   9504     681      3   36         11      964   False     True   
4   55.882   4897     357      2   68         16      331    True    False   

   Student_No  Student_Yes  Married_No  Married_Yes  Region_East  \
0        True        False       False         True        False   
1       False         True       False         True        False   
2        True        False        True        False        False   
3        True        False        True        False        False   
4        True        False       False         True        False   

   Region_South  Region_West  
0          True        False  
1         False         True  
2         False    

In [6]:
X = df.drop(columns=['Balance'])
y = df['Balance']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f'Train Data Shape (X, y): {X_train.shape, y_train.shape}')
print(f'Validation Data Shape (X, y): {X_val.shape, y_val.shape}')
print(f'Test Data Shape (X, y): {X_test.shape, y_test.shape}')

Train Data Shape (X, y): ((280, 15), (280,))
Validation Data Shape (X, y): ((60, 15), (60,))
Test Data Shape (X, y): ((60, 15), (60,))


In [8]:
# initialize the classifier models
dt_regressor = DecisionTreeRegressor(random_state=42)
xgb_regressor = XGBRegressor(random_state=42)

# initialize hyperparameters for gridsearchCV
dt_param_grid = {
    'max_depth': [2,3,4,5,8,10],
    'min_samples_split': [2,4,5,10],
    'min_samples_leaf': [1,3,5,8,10,12]
}

xgb_param_grid = {
    'max_depth': [1,3,5,7,10],
    'learning_rate': [0.1,0.01,0.001],
    'n_estimators': [10000,15000,16000],
    'subsample': [0.8,1.0]
}

In [9]:
# Decision Tree:
dt_grid_search = GridSearchCV(estimator=dt_regressor, param_grid=dt_param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
dt_grid_search.fit(X_train, y_train)

dt_best_params = dt_grid_search.best_params_
print(f"Decision Tree Best Parameters: {dt_best_params}")

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=4; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=4; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=4; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=10; total time=   0.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=10; total time=   0.0s
[CV] END max_depth=2, min_samples

In [10]:
# XGBoost:
xgb_grid_search = GridSearchCV(estimator=xgb_regressor, param_grid=xgb_param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
xgb_grid_search.fit(X_train, y_train)

xgb_best_params = xgb_grid_search.best_params_
print(f"XGBoost Best Parameters: {xgb_best_params}")

Fitting 3 folds for each of 90 candidates, totalling 270 fits
[CV] END learning_rate=0.1, max_depth=1, n_estimators=10000, subsample=0.8; total time=   2.3s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=10000, subsample=0.8; total time=   6.1s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=10000, subsample=0.8; total time=   1.8s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=10000, subsample=1.0; total time=   2.0s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=10000, subsample=1.0; total time=   2.2s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=10000, subsample=1.0; total time=   3.0s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=15000, subsample=0.8; total time=   7.0s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=15000, subsample=0.8; total time=   2.7s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=15000, subsample=0.8; total time=   2.7s
[CV] END learning_rate=0.1, max_depth=1, n_estimators=15000, subsample=1.0; total t

In [11]:
# now retrain the model on the combined training and validation set with the best hyperparameters
X_train_val = np.concatenate((X_train, X_val))
y_train_val = np.concatenate((y_train, y_val))

dt_final_model = DecisionTreeRegressor(**dt_best_params, random_state=42)
xgb_final_model = XGBRegressor(**xgb_best_params, random_state=42)

In [12]:
# retrain decision tree final model
dt_final_model.fit(X_train_val, y_train_val)

DecisionTreeRegressor(max_depth=10, min_samples_leaf=3, random_state=42)

In [13]:
# retrain xgboost final model
xgb_final_model.fit(X_train_val, y_train_val)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=1, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [14]:
# evaluate the final model on test set
y_test_pred_dt = dt_final_model.predict(X_test)
y_test_pred_xgb = xgb_final_model.predict(X_test)

In [20]:
def mse(y_actual, y_pred):
    error = 0.0

    for actual, predicted in zip(y_actual, y_pred):
        error += (actual - predicted) ** 2
    mse = error / len(y_actual)
    return mse

print(mse(y_test, y_test_pred_dt))
print(mse(y_test, y_test_pred_xgb))

17559.275226851856
3757.1172480234663
